In [1]:
import pandas as pd
import numpy as np
from DYNAMICS.dynamic_params import PARQUET_PATH, DB_PATH

In [2]:
PARQUET_PATH

'data/ohlc_5_min_test1_3330.parquet'

In [3]:
def load_data():
    return pd.read_parquet(PARQUET_PATH)

In [4]:
df = load_data()

In [5]:
df.head()

,timestamp,open,high,low,close,volume
0,2025-06-13 07:50:00+00:00,104742.1,104871.3,104742.0,104871.3,0.897758
1,2025-06-13 07:55:00+00:00,104871.2,104935.7,104871.2,104935.2,22.689674
2,2025-06-13 08:00:00+00:00,104935.3,105042.4,104935.3,105013.0,18.501232
3,2025-06-13 08:05:00+00:00,105012.9,105120.1,104949.1,105120.1,19.126656
4,2025-06-13 08:10:00+00:00,105121.5,105398.0,105121.4,105397.9,32.665882


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objs as go

def compute_sma(df: pd.DataFrame, period: int = 20) -> pd.DataFrame:
    """
    Compute the Simple Moving Average (SMA) and generate trading signals for a given DataFrame.

    Parameters:
      df : pd.DataFrame
          DataFrame containing at least the 'Close' price column.
      period : int, optional
          Window period to compute the moving average, default is 20.

    Returns:
      pd.DataFrame
          DataFrame with added columns:
              - 'SMA': The computed moving average.
              - 'Signal': Trading signal (1 for buy, -1 for sell) based on the price crossing above/below the SMA.
              - 'Position': Difference in signals indicating trade entries/exits.
              - 'Market_Return': Daily return of the underlying asset.
              - 'Strategy_Return': Return from the strategy calculated using a shifted signal.
              - 'Cumulative_Market': Cumulative return of the underlying asset.
              - 'Cumulative_Strategy': Cumulative return of the SMA strategy.
    """
    # Compute the SMA over the specified period
    df.loc[:, 'SMA'] = df['close'].rolling(window=period).mean()

    #df.loc[:, 'SMA'] = df.loc[:, 'SMA'].fillna(0)

        # Initialize the Signal column with zeros
    df.loc[:, 'Signal'] = 0

    # Generate signals only for the data points where SMA is available.
    # Signal = 1 when price > SMA, otherwise -1.
    df.loc[df.index[period:], 'Signal'] = np.where(
        df.loc[df.index[period:], 'close'] > df.loc[df.index[period:], 'SMA'],
        1,
        -1
    )

    # Calculate Position to indicate changes in signal (trade entries/exits)
    df.loc[:, 'Position'] = df['Signal'].diff()
    #df.loc[:, 'Position'] = df.loc[:, 'Position'].fillna(0)

    # Calculate daily market returns as percentage change
    df.loc[:, 'Market_Return'] = df['close'].pct_change()
    #df.loc[:, 'Market_Return'] = df.loc[:, 'Market_Return'].fillna(0)

    # Strategy return: we simulate execution based on the previous day's signal.
    df.loc[:, 'Strategy_Return'] = df['Market_Return'] * df['Signal'].shift(1)
    #df.loc[:, 'Strategy_Return'] = df.loc[:, 'Strategy_Return'].fillna(0)

    # Calculate cumulative returns for both market and strategy performance
    df.loc[:, 'Cumulative_Market'] = (1 + df['Market_Return']).cumprod()
    df.loc[:, 'Cumulative_Strategy'] = (1 + df['Strategy_Return']).cumprod()


    return df

In [7]:
data = compute_sma(df)

In [17]:
data["signal_change"] = data["Signal"].diff().fillna(0)

In [16]:
data.iloc[200:]

,timestamp,open,high,low,close,volume,SMA,Signal,Position,Market_Return,Strategy_Return,Cumulative_Market,Cumulative_Strategy,signal_change
200,2025-06-14 00:30:00+00:00,105834.5,105893.5,105834.4,105867.5,0.180576,105958.180,-1,0.0,0.000312,-0.000312,1.009499,0.997576,0.0
201,2025-06-14 00:35:00+00:00,105867.5,105903.6,105867.4,105867.5,0.357982,105961.205,-1,0.0,0.000000,-0.000000,1.009499,0.997576,0.0
202,2025-06-14 00:40:00+00:00,105867.5,105996.7,105867.4,105970.3,1.939164,105964.720,1,2.0,0.000971,-0.000971,1.010480,0.996607,2.0
203,2025-06-14 00:45:00+00:00,105970.3,106018.3,105957.8,106018.3,1.158148,105970.865,1,0.0,0.000453,0.000453,1.010937,0.997058,0.0
204,2025-06-14 00:50:00+00:00,106018.4,106172.3,105961.1,105961.1,6.886004,105977.400,-1,-2.0,-0.000540,-0.000540,1.010392,0.996521,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,2025-06-18 13:05:00+00:00,104400.1,104400.1,104262.9,104266.2,0.874554,104635.735,-1,0.0,-0.001363,0.001363,0.994230,1.022256,0.0
1504,2025-06-18 13:10:00+00:00,104266.3,104268.0,104100.0,104112.8,4.720377,104620.380,-1,0.0,-0.001471,0.001471,0.992767,1.023760,0.0
1505,2025-06-18 13:15:00+00:00,104112.7,104112.8,104000.0,104000.0,11.152379,104600.170,-1,0.0,-0.001083,0.001083,0.991692,1.024869,0.0
1506,2025-06-18 13:20:00+00:00,104000.1,104019.4,104000.0,104019.4,1.602582,104570.425,-1,0.0,0.000187,-0.000187,0.991877,1.024678,0.0
